# **Setup**

####**Import and install stuff (Takes a while)**

In [ ]:
# For google colab
!pip install transformers
!pip install clean-text
!pip install contractions
!pip install -U sentence-transformers
!pip install langid
!pip install argostranslate

# Run translation using GPU
#!ARGOS_DEVICE_TYPE=cuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 139.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.4 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=1989144bc4cc675402c94043c2d72d27aa165fa5bef5237eb348669ec0289398
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re
import ast
import string
import time
import spacy
import string
import contractions
import langid
import argostranslate.package
import argostranslate.translate
import sentence_transformers
import numpy as np
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from cleantext import clean
from transformers import pipeline

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download and install Argos Translate package
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
package_to_install = next(
    filter(
        lambda x: x.from_code == 'es' and x.to_code == 'en', available_packages
    )
)
argostranslate.package.install_from_path(package_to_install.download())

####**Load Dataset**

In [ ]:
github_url = "https://raw.githubusercontent.com/ExpiredTapWater/Datasets/main/Raw/Raw_Dataset_Combined.csv"
save_name = "Cleaned_Dataset_v2"

In [ ]:
data = pd.read_csv(github_url)
data

,ASIN,Product,Category,Price,URL,Reviews URL,Stars,Reviews
0,‎B01JPOLKDW,Logitech M330 Silent Plus Wireless Mouse – Enj...,Accessories,S$25.99,https://www.amazon.sg/Logitech-M330-Silent-Wir...,https://www.amazon.sg/Logitech-M330-Silent-Wir...,4.6,"[""The media could not be loaded.\n ..."
1,‎B0B11LJ69K,Logitech MX Master 3S Performance Wireless Mou...,Accessories,S$148.73,https://www.amazon.sg/Logitech-MX-Master-3S-Pe...,https://www.amazon.sg/Logitech-MX-Master-3S-Pe...,4.8,"['Good investment...', 'The roller-wheel broke..."
2,‎B07Y9PMKV2,"Wireless Keyboard and Mouse,JOYACCESS 2.4G Erg...",Accessories,S$35.99,https://www.amazon.sg/Wireless-Keyboard-JOYACC...,https://www.amazon.sg/Wireless-Keyboard-JOYACC...,4.2,['Quiet nice smooth to use. Worth it. Sleek mo...
3,‎B01MQ2A86A,Logitech M331 Silent Plus Wireless Mouse 2.4GH...,Accessories,S$23.90,https://www.amazon.sg/Logitech-Silent-Wireless...,https://www.amazon.sg/Logitech-Silent-Wireless...,4.5,['Amazing Amazon 1 day delivery !Product is as...
4,‎B09QXCWLNS,Logitech M650 L Signature Wireless Mouse Graph...,Accessories,S$42.83,https://www.amazon.sg/Logitech-Signature-Wirel...,https://www.amazon.sg/Logitech-Signature-Wirel...,4.5,['Delivery was on time well packed and great p...
...,...,...,...,...,...,...,...,...
5266,‎B00CRJWUHI,"CRU-Wiebetech 6417-6500-0500 Data Express, Sto...",Storage,S$112.88,https://www.amazon.sg/CRU-Wiebetech-6417-6500-...,No Reviews,No Reviews,No Reviews
5267,‎B0077COX8E,Centon Electronics Direct 2.5-Inch Solid State...,Storage,S$51.27,https://www.amazon.sg/Centon-Electronics-Direc...,https://www.amazon.sg/Centon-Electronics-Direc...,3.1,['PROS:- Way faster than a HDD.- 480GB capacit...
5268,‎B002M51BF4,Mita Sangyo SSD Phantom 32GB (MLC) Lead: 170MB...,Storage,S$53.58,https://www.amazon.sg/Mita-Sangyo-Phantom-32GB...,No Reviews,No Reviews,No Reviews
5269,‎B002FKTFUO,Silicone Power SP064GBSSDJ10C00 Portable Solid...,Storage,S$200.76,https://www.amazon.sg/Silicone-Power-SP064GBSS...,https://www.amazon.sg/Silicone-Power-SP064GBSS...,3.0,['細身で持ち運びには最適ですが、パワーeSATAだということに気がつきませんでした。確認す...


# **Basic Dataset Cleaning**

####**Check for missing values**

In [ ]:
# Drop reviews URL as I only used that for debug
data = data.drop(['Reviews URL'], axis=1)

In [ ]:
# Check for missing values (Nan)
data.isnull().sum()

ASIN         0
Product      2
Category     0
Price       82
URL          0
Stars       90
Reviews      0
dtype: int64

In [ ]:
# Drop rows with missing values
data.dropna(inplace = True)

# Verify changes
data.isnull().sum()

ASIN        0
Product     0
Category    0
Price       0
URL         0
Stars       0
Reviews     0
dtype: int64

In [ ]:
# Check for dataset specific missing values
def verify():
  rows = len(data)

  try:
      asin_err = data['ASIN'].value_counts()['No ASIN']
  except KeyError:
      asin_err = 0
  asin_err += data['ASIN'].isna().sum()

  try:
      price_err = data['Price'].value_counts()['Error']
  except KeyError:
      price_err = 0
  price_err += data['Price'].isna().sum()

  try:
      reviews_err = data['Reviews'].value_counts()['No Reviews']
  except KeyError:
      reviews_err = 0
  reviews_err += data['Price'].isna().sum()

  try:
      stars_err = data['Stars'].value_counts()['No Reviews']
  except KeyError:
      stars_err = 0
  stars_err += data['Stars'].isna().sum()

  print(f"No ASIN: {asin_err}({data['ASIN'].isna().sum()}) of {rows}")
  print(f"No Price: {price_err}({data['Price'].isna().sum()}) of {rows}")
  print(f"No Reviews: {reviews_err}({data['Reviews'].isna().sum()}) of {rows}")
  print(f"No Stars: {stars_err}({data['Stars'].isna().sum()}) of {rows}")

verify()

No ASIN: 50(0) of 5181
No Price: 0(0) of 5181
No Reviews: 990(0) of 5181
No Stars: 369(0) of 5181


In [ ]:
# Drop rows with missing values
data.drop(data[data['ASIN'] == "No ASIN"].index, inplace = True)
data.drop(data[data['Price'] == "Error"].index, inplace = True)
data.drop(data[data['Reviews'] == "No Reviews"].index, inplace = True)
data.drop(data[data['Stars'] == "No Reviews"].index, inplace = True)

data.drop(data[data['Reviews'] == "No Reviews Available"].index, inplace = True)
data.drop(data[data['Stars'] == "No Reviews Available"].index, inplace = True)

# Verify changes
verify()

No ASIN: 0(0) of 4128
No Price: 0(0) of 4128
No Reviews: 0(0) of 4128
No Stars: 0(0) of 4128


####**Check for duplicates**

In [ ]:
# Check for duplicates products
print(data['ASIN'].duplicated().value_counts())
print(data['Product'].duplicated().value_counts())

False    4110
True       18
Name: ASIN, dtype: int64
False    4103
True       25
Name: Product, dtype: int64


In [ ]:
# Take a peek before deciding to drop or not
data[data['ASIN'].duplicated(keep=False) == True].sort_values('ASIN')

,ASIN,Product,Category,Price,URL,Stars,Reviews
1503,‎4328463797,White Label 120GB 8MB Cache 7200RPM PATA/IDE 3...,Storage,S$72.60,https://www.amazon.sg/White-Label-7200RPM-Desk...,3.9,['メルカリで買った動作保証なし中古のIDE hddが動かなかったので、本品を購入しました。...
1514,‎4328463797,White Label 120GB 8MB Cache 7200RPM PATA/IDE 3...,Storage,S$72.60,https://www.amazon.sg/White-Label-7200RPM-Desk...,3.9,['メルカリで買った動作保証なし中古のIDE hddが動かなかったので、本品を購入しました。...
1416,‎8541578856,White Label 2 Terabyte (2TB) 16MB Cache 7200RP...,Storage,S$163.36,https://www.amazon.sg/White-Label-Terabyte-720...,4.1,['Solo fué instalarlo en mi DVR y arrancó sin ...
1415,‎8541578856,White Label 2 Terabyte (2TB) 16MB Cache 7200RP...,Storage,S$163.36,https://www.amazon.sg/White-Label-Terabyte-720...,4.1,['Solo fué instalarlo en mi DVR y arrancó sin ...
1509,‎B004K6QZLI,IBM 600GB 10000 RPM SAS 6Gbps 2.5-Inch 600 SAS...,Storage,S$596.55,https://www.amazon.sg/IBM-2-5-Inch-Internal-Dr...,5.0,['Everything perfect']
1524,‎B004K6QZLI,IBM 600GB 10000 RPM SAS 6Gbps 2.5-Inch 600 SAS...,Storage,S$596.55,https://www.amazon.sg/IBM-2-5-Inch-Internal-Dr...,5.0,['Everything perfect']
1515,‎B006FRDSJC,"BUFFALO 3.5"" Serial ATA Internal HDD 1TB HD-ID...",Storage,S$116.79,https://www.amazon.sg/BUFFALO-3-5-Serial-Inter...,3.8,['以前のHDDの調子が悪くなり、交換しました。乗り換えも、付属のソフトでスムーズに出来まし...
1504,‎B006FRDSJC,"BUFFALO 3.5"" Serial ATA Internal HDD 1TB HD-ID...",Storage,S$116.79,https://www.amazon.sg/BUFFALO-3-5-Serial-Inter...,3.8,['以前のHDDの調子が悪くなり、交換しました。乗り換えも、付属のソフトでスムーズに出来まし...
1501,‎B0086ZUP1A,BUFFALO 500GB OP-HD500/4K Terra Station/Link S...,Storage,S$181.49,https://www.amazon.sg/BUFFALO-OP-HD500-4K-Stat...,3.8,"['ありがとうございました。', '又すぐ壊れたので、本体ごと捨てました。', '突然、仕事..."
1512,‎B0086ZUP1A,BUFFALO 500GB OP-HD500/4K Terra Station/Link S...,Storage,S$181.49,https://www.amazon.sg/BUFFALO-OP-HD500-4K-Stat...,3.8,"['ありがとうございました。', '又すぐ壊れたので、本体ごと捨てました。', '突然、仕事..."


**Rows contain duplicate reviews, so we will drop them**

In [ ]:
# Drop rows
data.drop_duplicates(subset=['ASIN'], inplace = True)
data.drop_duplicates(subset=['Product'], inplace = True)

# Verify changes
verify()

No ASIN: 0(0) of 4099
No Price: 0(0) of 4099
No Reviews: 0(0) of 4099
No Stars: 0(0) of 4099


**Convert reviews back to lists due to importing from csv interpreting as string**

In [ ]:
def str_to_list():
  for i in data.index:
    data['Reviews'][i] = ast.literal_eval(data['Reviews'][i])

str_to_list()

In [ ]:
# Verify
type(data.iloc[0, 6])

list

#**Dataset Specific Cleaning**

#### **Remove emojis**

In [ ]:
# Create a simple function
def remove_emojis(text):
  output = []
  for i in text:
    output.append(clean(i, no_emoji = True, lower = False))
    #print(i)
  return output

# Create test entry to verify removal
data.iloc[-1] = ["", "", "",  "",  "",  "", ["This😅 text I'll 😃🤣 contains 😁 😆😄 emojis   😂 "] ]
data.tail(1)

,ASIN,Product,Category,Price,URL,Stars,Reviews
5269,,,,,,,[This😅 text I'll 😃🤣 contains 😁 😆😄 emojis 😂 ]


In [ ]:
# Apply function to dataset
data['Reviews'] = data['Reviews'].apply(lambda x: remove_emojis(x))

# Verify
data.tail(1)

,ASIN,Product,Category,Price,URL,Stars,Reviews
5269,,,,,,,[This text I'll contains emojis]


In [ ]:
# Drop test entry
data.drop(data.index[-1], inplace = True)

# Verify
data.tail(1)

,ASIN,Product,Category,Price,URL,Stars,Reviews
5267,‎B0077COX8E,Centon Electronics Direct 2.5-Inch Solid State...,Storage,S$51.27,https://www.amazon.sg/Centon-Electronics-Direc...,3.1,[PROS:- Way faster than a HDD.- 480GB capacity...


In [ ]:
# Check if still list
type(data.iloc[0, 6])

list

####**Translate foriegn reviews (Takes a while)**

*Google's googletrans package is the simplest to implement, but requires the use of external API to return results. Free users will get blocked for too many requests*

* *The solution was to implement langid, which can detect the language but cannot translate text, and argos translator, which requires the origin text to be already be language identified before translation. Both work locally*

In [ ]:
# Translation function using argos tranaslate
def translate(text, origin_language):
  translation = argostranslate.translate.translate(text, origin_language, 'en')
  return translation

In [ ]:
# Use langid to identify language
english_reviews = 0
foriegn_reviews = 0

# Iterate over index
for i in data.index:
  output = []
  print(f"{i} of {len(data.index)}")

  # Reviews are stored as a list
  for review in data['Reviews'][i]:

    # Identify language using Langid. Element 0 represents text return eg. 'en'
    language = langid.classify(review)[0]

    # Nothing to do if the review is already in English
    if language == 'en':
      output.append(review)
      english_reviews += 1

    # If review is not English
    else:
      try:
        # Translate to english
        translation = translate(review, language)

      # Catch caseses of mis-formatted strings that slipped past
      except AttributeError:
        pass

      # Append translated review
      else:
        output.append(translation)
        foriegn_reviews += 1

    # Save list of translated reviews
    data['Reviews'][i] = output

0 of 4098
1 of 4098
2 of 4098
3 of 4098
4 of 4098
5 of 4098
6 of 4098
7 of 4098
8 of 4098
9 of 4098
10 of 4098
11 of 4098
12 of 4098
13 of 4098
14 of 4098
15 of 4098
16 of 4098
17 of 4098
18 of 4098
19 of 4098
20 of 4098
21 of 4098
22 of 4098
23 of 4098
24 of 4098
25 of 4098
26 of 4098
28 of 4098
29 of 4098
30 of 4098
31 of 4098
32 of 4098
33 of 4098
34 of 4098
35 of 4098
36 of 4098
37 of 4098
38 of 4098
39 of 4098
40 of 4098
41 of 4098
42 of 4098
43 of 4098
44 of 4098
45 of 4098
46 of 4098
49 of 4098
50 of 4098
51 of 4098
52 of 4098
53 of 4098
55 of 4098
56 of 4098
57 of 4098
58 of 4098
59 of 4098
60 of 4098
61 of 4098
62 of 4098
63 of 4098
64 of 4098
65 of 4098
66 of 4098
67 of 4098
68 of 4098
69 of 4098
70 of 4098
71 of 4098
72 of 4098
74 of 4098
75 of 4098
76 of 4098
78 of 4098
79 of 4098
80 of 4098
81 of 4098
83 of 4098
84 of 4098
85 of 4098
86 of 4098
87 of 4098
88 of 4098
90 of 4098
92 of 4098
93 of 4098
95 of 4098
96 of 4098
97 of 4098
98 of 4098
99 of 4098
100 of 4098
101 of 4

KeyboardInterrupt: ignored

In [ ]:
# View statistics

print(f"\nEnglish: {english_reviews}")
print(f"Foriegn: {foriegn_reviews}")
print(f"Total Reviews: {english_reviews+foriegn_reviews} ")

In [ ]:
# Verify
data#.sample(20)

# **NLP Preprocessing**

#### **Tasks Overview**






**Basic Cleaning (Previous section):**
* Removing null values
* Removing duplicates
* Convert str back to list

**Dataset Specific Cleaning**
* Remove emojis
* Translate all text to english

**NLP Specific Cleaning:**
* Remove hyperlinks
* Expanding contractions
* Removing punctuations
* Lemmatizing text
* Lowercasing
* Remove Numbers
* Removing stopwords
* Remove leftover characters
* Tokenize (If not using pipeline)
* Text Representation

**Notes on sequence:**
* Expand contractions first before removing punctuations because you need apostrophe to expand
* Convert emoticons and emoji first before removing punctuations else it won't work (eg :-D)
* Remove non ASCII after expaning emojis
* Lowercase after emoji (:D to :d doesnt make sense)

####**Create new columns to perform NLP text preprocessing seperate from the actual reviews**

In [ ]:
data['NLP_Reviews'] = data.loc[:, 'Reviews']
data.sample(10)

**Check if NLP_Reviews is a list an not str due to loading from CSV**

In [ ]:
print(f"This should output list: {type(data.iloc[0, 6])}")

**Create empty columns for NLP results**

In [ ]:
data["Embeddings"] = np.nan
data["Summary"] = np.nan
data["Sentiment"] = np.nan
data.head(5)

####**Remove hyperlinks**

In [ ]:
# Create test entry to verify removal
data.iloc[-1] = ["", "", "", "", "", "", "","link: https://www.amazon.sg/QuietComfort-Wireless-Cancelling-Headphones-Bluetooth/dp/B0CCZ1SQ8G/ End of text","","","" ]
data.tail(1)

In [ ]:
# Remove links
data['NLP_Reviews'].replace(r'http\S+','', regex = True, inplace = True)

# Verify
data.tail(1)

In [ ]:
# Drop test entry
data.drop(data.index[-1], inplace = True)

# Verify
data.tail(1)

####**Expand Contractions**
*Contractions are words or combinations of words that are shortened by dropping letters and replacing them by an apostrophe*

In [ ]:
for i in data.index:
  output = []
  for reviews in data['NLP_Reviews'][i]:
    for word in reviews.split():
      output.append(contractions.fix(word))

  data['NLP_Reviews'][i] = ' '.join(output)

In [ ]:
# Verify
data['NLP_Reviews'].sample(20)

####**Remove Punctuations**

In [ ]:
# Create small function
def remove_punctuation(text):
    return text.translate(str.maketrans('','',string.punctuation))

# Apply
data['NLP_Reviews'] = data['NLP_Reviews'].apply(lambda x: remove_punctuation(x))

In [ ]:
# Verify
data['NLP_Reviews'].sample(20)

####**Lemmantize Text**
*Lemmatization is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item*

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Code referenced from: https://spacy.io/models/en#en_core_web_md
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [ ]:
# Apply
data['NLP_Reviews'] = data['NLP_Reviews'].apply(lambda x:lemmatize_text(x))

In [ ]:
# Verify
data['NLP_Reviews'].sample(20)

####**Lowercase**

In [ ]:
# Use built in lower() function
data['NLP_Reviews'] = data['NLP_Reviews'].apply(lambda x: x.lower())

In [ ]:
# Verify
data['NLP_Reviews'].sample(20)

####**Remove Numbers**

In [ ]:
# Use built in string library to remove stopwords
mapping = str.maketrans('', '', string.digits)

for i in data.index:
  data['NLP_Reviews'][i] = data['NLP_Reviews'][i].translate(mapping)

In [ ]:
# Verify
data

####**Remove Stopwords**
*They are often considered noise in text analysis tasks because they do not carry significant information about the content*

In [ ]:
# Use spaCy to remove stopwords
for i in data.index:

    # Process text
    doc = nlp(data['NLP_Reviews'][i])

    # iterate over each word and check if is a stopword from imported stopwords
    filtered_text = [token.text for token in doc if not token.is_stop]

    # Save to dataframe
    data['NLP_Reviews'][i] = ' '.join(filtered_text)

In [ ]:
# Verify
data

####**Remove Single Characters & White Spaces**

In [ ]:
# Removes single character words, and any extra whitespaces leftover from formatting
for i in data.index:

  # Convert string of text to list
  all_texts = data['NLP_Reviews'][i].split(" ")
  output = ""

  # Iterate over each element in list
  for text in all_texts:

    # Add text to output if its length is more than 1
    if not len(text) <= 1:
      output += (text + " ")

  # Save
  data['NLP_Reviews'][i] = output

In [ ]:
# Verify
data

####**Text Representation**

*Encodes words into numbers for processing. We will use the ["all-MiniLM-L6-v2"](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) model from hugging face, the most popular lightweight model from their community. Unlike other methods of encoding eg. BoW, one-hot, etc, this one will tokenize for us.*


In [ ]:
# Load sentence transformer
model = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Save to dataframe embeddings
for i in data.index:
  data['Embeddings'][i] = model.encode(data['NLP_Reviews'][i])

#**Save Dataset**

**Final Cleanup**

In [ ]:
data.drop(data[data['NLP_Reviews'] == ""].index, inplace = True)

In [ ]:
# Verify is list
type(data['Reviews'][0])

list

**Preview one last time before saving**

In [ ]:
data.sample(20)

**Save**

In [ ]:
data.to_pickle(f'{save_name}.pkl')
data.to_csv(f'{save_name}.csv')

#**NLP Procesing**

####**Load Half Completed Dataset (Skip Preprocessing)**
*Import as pickle file!*

In [ ]:
#github_url = "https://raw.githubusercontent.com/ExpiredTapWater/Datasets/main/Cleaned_Dataset.pkl"
#data = pd.read_pickle(github_url)

####**Summarize Reviews**
*Most popular text summarization model on Hugging Face, very slow processing ~2H*

In [ ]:
# Download model and create pipeline (Don't Tokenize)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
# Debug
count = 0
rows = len(data.index)

# Iterate over each row
for i in data.index:

  reviews = data['Reviews'][i]

  # Join all list of sentences together, then split to get individual words
  all_words = " ".join(reviews)
  word_list = all_words.split(" ")

  # Debug
  print(f"[#] {count} of {rows} [word_list] {len(word_list)}", end="\r")
  count += 1

  # Ensure word count does not exceed model limit of 512 tokens for BERT models
  if len(word_list) > 512:
      word_list = word_list[:512]
      all_words = " ".join(word_list)

  # Keep original if word count too little
  if len(word_list) < 10:
      data['Summary'][i] = reviews
      continue

  # Summarize review, save only the output text found at [0]['summary_text']
  try:
      output = summarizer(all_words, min_length=8, max_length = 40)[0]['summary_text']

  # Catch some weirdly formatted fields
  except IndexError:
      output = reviews

  data['Summary'][i] = output

####**Sentiment Analysis**
*Fine tuned model trained on Amazon reviews dataset*

In [ ]:
# Download model and create pipeline (Don't Tokenize)
sentiment_model = pipeline("sentiment-analysis", model ="Kaludi/Reviews-Sentiment-Analysis" )

In [ ]:
# Debug
count = 0
rows = len(data.index)

# Create extra column that I forgot to in the begining
data["Confidence"] = np.nan

# Iterate over each row on pre-formatted NLP_Reviews column
for i in data.index:


  out = ""

  # Format stored string of words into list of words
  reviews_string = list(data['NLP_Reviews'][i].split(" "))

  # Slice word count exceeds limit
  if len(reviews_string) > 512:
    reviews_string = reviews_string[:512]
    out = " ".join(reviews_string)

  else:
    out = " ".join(reviews_string)

  try:
     # Save to sentiment column only the label (positive, negative)
    results = sentiment_model(out)[0]
    data['Sentiment'][i] = results['label']
    data['Confidence'][i] = results['score']

  # Catch some weirdly formatted fields
  except IndexError:
    data['Sentiment'][i] = 'ERROR'

  # Debug
  print(f"[Index] {count} of {count} [len(reviews_string)] {len(reviews_string)}", end="\r")
  count += 1

In [ ]:
# Verify
data['Sentiment'].value_counts()

#**Post-Processing**

####**Clean up any errors from NLP**

In [ ]:
# Count number of rows with error
def check():
    count = 0

    # Those with error are in this format '[ERROR]'

    # Iterate over string
    for i in data.index:
        test = data['Summary'][i][0]

        # Check if first char is '['
        if test == '[':
            count += 1

    print(f"Error Count: {count}")
check()

In [ ]:
# Resolve issue
count=0

for i in data.index:
    test = data['Summary'][i][0]
    if test == '[':

      # Drop rows with error
      data.drop(index = i, inplace=True)
      count += 1

print(f"Errors Removed: {count}")

check()

**Format back from str to list if imported as CSV before saving file**

In [ ]:
str_to_list()

In [ ]:
data.to_pickle(f'{save_name}.pkl')
data.to_csv(f'{save_name}.csv')